In [1]:
import json

import scipy as sp
import pandas as pd
import numpy as np

from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from collections import Counter

import lightgbm as lgb
np.random.seed(369)

from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm, tqdm_notebook

In [2]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = quadratic_weighted_kappa(y, X_p)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
def rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

### Image Features ###

In [3]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook
from keras.applications.densenet import preprocess_input, DenseNet121
import tensorflow as tf

train_df = pd.read_csv('input/train/train.csv')
img_size = 256
batch_size = 16

Using TensorFlow backend.


In [4]:
pet_ids = train_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

In [5]:
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

In [6]:
features = {}

def train_keras():
    from keras.models import Model
    from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
    from keras import backend as K
    import tensorflow as tf

    #https://regressionsessionsblog.wordpress.com/2018/06/11/stuck-on-an-issue-making-keras-predictions-in-parallel/
    #https://github.com/keras-team/keras/issues/4740
    jobs = 6
    config = tf.ConfigProto(intra_op_parallelism_threads=jobs, \
                            inter_op_parallelism_threads=jobs, \
                            allow_soft_placement=True, \
                            device_count = {'CPU': jobs})
    session = tf.Session(config=config)
    K.set_session(session)

    print("start")
    inp = Input((256,256,3))
    backbone = DenseNet121(input_tensor = inp, 
                            weights="input/densenet-keras/DenseNet-BC-121-32-no-top.h5",
                            include_top = False)
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
    x = AveragePooling1D(4)(x)
    out = Lambda(lambda x: x[:,:,0])(x)

    m = Model(inp,out)
    print("done")

    

    print("start keras prediction")

    for b in tqdm_notebook(range(n_batches)):
        start = b*batch_size
        end = (b+1)*batch_size
        batch_pets = pet_ids[start:end]
        batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
        for i,pet_id in enumerate(batch_pets):
            try:
                batch_images[i] = load_image("input/train/train_images/", pet_id)
            except:
                pass
        batch_preds = m.predict(batch_images)
        for i,pet_id in enumerate(batch_pets):
            features[pet_id] = batch_preds[i]
    print("done keras prediction")
    return m

In [7]:
from keras.models import Model
m = train_keras()

start
done
start keras prediction



done keras prediction


In [18]:
def save_json(features, filename="train_features"):
    with open(filename + ".json", 'w') as file:
        json.dump(features,file)
        #json.dump(dictionary, file, sort_keys=True, indent=4)
    return
def load_json(filename="file", dictionary=features):
    with open(filename + ".json", 'r') as file:
        loaded_file = json.load(file)
    return loaded_file

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(features, cls=NumpyEncoder)

with open("train_features.json", 'w') as f:
    json.dump(dumped, f)
    print("Saved train features json to disk")

In [32]:

#print(m.summary())

m.save_weights("keras_model_weights.h5")
print("Saved model weights to disk")
#with open("train_features.json", 'r') as f:
#    train_loaded_features = json.load(f)
#    train_loaded_features = json.loads(train_loaded_features)
#    print("loaded train features")
    

Saved model weights to disk


In [35]:
train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats.columns = ['pic_'+str(i) for i in range(train_feats.shape[1])]
test_df = pd.read_csv('input/test/test.csv')

pet_ids = test_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1
test_features = {}

def train_test_keras(m, n_batches):
    from keras.models import Model
    from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
    from keras import backend as K
    import tensorflow as tf

    #https://regressionsessionsblog.wordpress.com/2018/06/11/stuck-on-an-issue-making-keras-predictions-in-parallel/
    #https://github.com/keras-team/keras/issues/4740
    jobs = 6
    config = tf.ConfigProto(intra_op_parallelism_threads=jobs, \
                            inter_op_parallelism_threads=jobs, \
                            allow_soft_placement=True, \
                            device_count = {'CPU': jobs})
    session = tf.Session(config=config)
    K.set_session(session)

    print("start create model")
    inp = Input((256,256,3))
    backbone = DenseNet121(input_tensor = inp, 
                            weights="input/densenet-keras/DenseNet-BC-121-32-no-top.h5",
                            include_top = False)
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
    x = AveragePooling1D(4)(x)
    out = Lambda(lambda x: x[:,:,0])(x)

    m = Model(inp,out)
    print("done")
    
    print("start keras test prediction")

    for b in tqdm_notebook(range(n_batches)):
        start = b*batch_size
        end = (b+1)*batch_size
        batch_pets = pet_ids[start:end]
        batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
        for i,pet_id in enumerate(batch_pets):
            try:
                batch_images[i] = load_image("input/test/test_images/", pet_id)
            except:
                pass
        batch_preds = m.predict(batch_images)
        for i,pet_id in enumerate(batch_pets):
            test_features[pet_id] = batch_preds[i]
    print("done keras test prediction")
    return

train_test_keras(m, n_batches)

dumped = json.dumps(test_features, cls=NumpyEncoder)

with open("test_features.json", 'w') as f:
    json.dump(dumped, f)
    print("Saved test features json to disk")

start create model
done
start keras test prediction



done keras test prediction
Saved test features json to disk


In [43]:
test_feats = pd.DataFrame.from_dict(features, orient='index')
test_feats.columns = ['pic_'+str(i) for i in range(test_feats.shape[1])]
test_feats = test_feats.reset_index()
test_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

train_feats = train_feats.reset_index()
train_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

test_feats.head()

test_feats.to_csv(r'csv_out/img_test_feats_prediction_values.csv')
print("saved image test features prediction values to CSV")

saved image test features prediction values to CSV


In [58]:
train = pd.read_csv("input/train/train.csv")
test = pd.read_csv("input/test/test.csv")

target = train['AdoptionSpeed']
train_id = train['PetID']
test_id = test['PetID']

loaded_train_feats = {}
loaded_test_feats = {}
with open("train_features.json", 'r') as f:
    loaded_train_feats = json.load(f)
    loaded_train_feats = json.loads(loaded_train_feats)
    print("loaded train features")

with open("test_features.json", 'r') as f:
    loaded_test_feats = json.load(f)
    loaded_test_feats = json.loads(loaded_test_feats)
    print("loaded test features") 

train_feats = pd.DataFrame.from_dict(loaded_train_feats, orient='index')
train_feats.columns = ['pic_'+str(i) for i in range(train_feats.shape[1])]

test_feats = pd.DataFrame.from_dict(loaded_test_feats, orient='index')
test_feats.columns = ['pic_'+str(i) for i in range(test_feats.shape[1])]

train_feats = train_feats.reset_index()
train_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)
test_feats = test_feats.reset_index()
test_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)


train = train.join(train_feats.set_index('PetID'),on='PetID')
test = test.join(test_feats.set_index('PetID'),on='PetID')
#train = pd.merge(train, train_feats, on = ['PetID'], how = 'left')
#test = pd.merge(test, test_feats, left_on = ['PetID'], right_on = ['PetID'], how = 'outer')

train.drop(['AdoptionSpeed', 'PetID'], axis=1, inplace=True)
test.drop(['PetID'], axis=1, inplace=True)

train.to_csv(r'csv_out/train_csv_img_merge.csv')
print("saved train csv image merge")
train.to_csv(r'csv_out/test_csv_img_merge.csv')
print("saved test csv image merge")

loaded train features
loaded test features
saved train csv image merge
saved test csv image merge
